In [162]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding,Activation, merge


In [163]:
data = pd.read_csv('./data/Tweets.csv')  


In [164]:
data.shape

(14640, 15)

In [165]:
Conf = data['airline_sentiment_confidence']

In [166]:
Conf.shape

(14640,)

In [167]:
data = data[data['airline_sentiment'] != 'neutral' ]
Conf = data['airline_sentiment_confidence']
Xtrain = data['text']
Ytrain= data['airline_sentiment']


In [135]:
data['airline_sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [ ]:
import pixiedust
from IPython.core.debugger import set_trace
def cleaning(data):
    data = data.replace("\\d","")
#     set_trace()
    word = [word for word in data.split()
            if 'http' not in word
            and not word.startswith('@') 
            and not word.startswith('$')             
           ]
    return " ".join(word)
Xtrain= Xtrain.apply(cleaning)

In [137]:
# XTrain = Xtrain['text']
# Xtrain['text'].shape

In [138]:
embedding_matrix = dict()
f = open('.././Glove/Glove+Wiki/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:],dtype='float32')
    embedding_matrix[word] = coefs
f.close()

In [139]:
t = Tokenizer()
t.fit_on_texts(Xtrain)
encoded_docs = t.texts_to_sequences(Xtrain)
padded = pad_sequences(encoded_docs, maxlen = 175, padding='post')

In [140]:
vocabsize = len(t.word_index) + 1
embeddedmatrix = zeros((vocabsize, 100))
for word, i in t.word_index.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None :
        embeddedmatrix[i] = embedding_vector
def sentiment(data):
    if data == 'negative':
        return 0
    else :
        return 1

Ytrain = Ytrain.apply(sentiment)


In [153]:
model = Sequential()
model.add(Embedding(vocabsize, 100, weights=[embeddedmatrix], input_length=175, trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [154]:
model.fit(padded,Ytrain)

Epoch 1/1
11541/11541 [==============================] - 3s 291us/step - loss: 0.3677 - acc: 0.8493


In [147]:
model = Sequential()
model.add(Dense(1, input_dim = 1))
model.add(Activation('relu'))

# # model_final = Graph() 
# model_final.add(merge([model_query, model], mode='concat'))
# model_final.add(Dropout(0.5))
# model_final.add(Dense(1))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [159]:
model = Sequential()
model.add(Embedding(vocabsize, 100, weights=[embeddedmatrix], input_length=175, trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


# model = Sequential()
# model.add(Dense(1, input_dim = 1))
# model.add(Activation('relu'))



In [161]:
from keras.layers import Dropout, Input
model1 = Input((1,))
model1 = Dense(1)(model)
model1 = Activation('relu')(model)


ValueError: Layer dense_47 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.engine.sequential.Sequential'>. Full input: [<keras.engine.sequential.Sequential object at 0x00000223B86A8588>]. All inputs to the layer should be tensors.

In [145]:
ab = merge([],mode='concat')

(11541,)

In [148]:
model.fit(Conf,Ytrain)

Epoch 1/1
11541/11541 [==============================] - 4s 314us/step - loss: 3.3002 - acc: 0.7953
